In [1]:
from datetime import datetime
from dateutil.relativedelta import relativedelta

import json
import os
import pandas as pd
pd.set_option("display.max_columns", 30)

import requests

In [2]:
"""
1. get the data from s3
2. weather data transformations
3. taxi trips transformations - DONE
4. update payment_type_master - DONE
5. update company_master - DONE
6. update taxi_trips with company and payment_type ids (replace the string values with ids from the latest master tables) - DONE
7. upload weather data to s3
8.upload taxi data to s3
9. upload the newest payment_type_master and company_master

"""

'\n1. get the data from s3\n2. weather data transformations\n3. taxi trips transformations - DONE\n4. update payment_type_master - DONE\n5. update company_master - DONE\n6. update taxi_trips with company and payment_type ids (replace the string values with ids from the latest master tables) - DONE\n7. upload weather data to s3\n8.upload taxi data to s3\n9. upload the newest payment_type_master and company_master\n\n'

### Taxi_trips transformation codes

In [4]:
current_datetime = datetime.now() - relativedelta(months=2)
formatted_datetime = current_datetime.strftime("%Y-%m-%d")

url = f"https://data.cityofchicago.org/resource/ajtu-isnz.json?$where=trip_start_timestamp >= '{formatted_datetime}T00:00:00' AND trip_start_timestamp <= '{formatted_datetime}T23:59:59'&$limit=30000"

headers = {"X-App-Token": os.environ.get("CHICAGO_API-TOKEN")}

response = requests.get(url)

data = response.json()


In [5]:
taxi_trips = pd.DataFrame(data)

In [6]:
taxi_trips.drop(["pickup_census_tract", "dropoff_census_tract"], axis=1, inplace=True)
taxi_trips.drop(["pickup_centroid_location", "dropoff_centroid_location"], axis=1, inplace=True)

taxi_trips.dropna(inplace=True)

taxi_trips.rename(columns={"pickup_community_area": "pickup_community_area_id",
                           "dropoff_community_area": "dropoff_community_area_id" }, inplace=True)

taxi_trips["datetime_for_weather"] = pd.to_datetime(taxi_trips["trip_start_timestamp"]).dt.floor("h")


In [7]:
taxi_trips.head()

,trip_id,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_community_area_id,dropoff_community_area_id,fare,tips,tolls,extras,trip_total,payment_type,company,pickup_centroid_latitude,pickup_centroid_longitude,dropoff_centroid_latitude,dropoff_centroid_longitude,datetime_for_weather
0,eebf0bf993bd8f0be7df4df32f95bd9bdcdcf235,6b00c6e07523bf0964046af1bf69575f25c6ea393f288d...,2024-10-19T23:45:00.000,2024-10-20T00:30:00.000,1904,18,76,28,44.25,9.95,0,5,59.7,Credit Card,Flash Cab,41.980264315,-87.913624596,41.874005383,-87.66351755,2024-10-19 23:00:00
2,ebb79136e351be12b719c769fa671f625a86b523,99ec13d5d806f5f5fa7a57910f8e38d84f90630529f2f8...,2024-10-19T23:45:00.000,2024-10-19T23:45:00.000,7,0,24,24,25,3.82,0,0,29.32,Credit Card,Flash Cab,41.901206994,-87.676355989,41.901206994,-87.676355989,2024-10-19 23:00:00
3,0040b32624953b9e5435e3d4b4ebeb68e81e45f0,1e4ba7f6a2c79ac2220073aa7006dd757feed6b12d4a40...,2024-10-19T23:45:00.000,2024-10-19T23:45:00.000,435,1.02,8,8,6.5,3,0,1,11,Credit Card,Sun Taxi,41.899602111,-87.633308037,41.899602111,-87.633308037,2024-10-19 23:00:00
4,0745379149f8b92e3e663f966307da211ccdf689,4ba3690e7f7bde263c96de93e97af2af2545f6d16f06b7...,2024-10-19T23:45:00.000,2024-10-20T00:15:00.000,1504,12.83,33,1,40.83,0,0,0,40.83,Cash,Flash Cab,41.857183858,-87.620334624,42.009622881,-87.670166857,2024-10-19 23:00:00
5,082d93dd7db2895533aac917fd42011fcf7f5c12,fe806ca7a45786db86ed61fe4c027b1f5a5a4b5c9f3595...,2024-10-19T23:45:00.000,2024-10-19T23:45:00.000,405,1.3,8,8,6.75,0,0,0,6.75,Cash,Chicago Independents,41.899602111,-87.633308037,41.899602111,-87.633308037,2024-10-19 23:00:00


#### Taxi_trips transformation function

In [8]:
def taxi_trips_transformations(taxi_trips: pd.DataFrame) -> pd.DataFrame:
    """ Perform transformation with the taxi data.

    Parameters
    ----------
    taxi_trips : pd.Dataframe
        The Dataframe holding the daily taxi trips

    Returns
    ------
    pd.DataFrame
        The cleaned, transformed DataFrame holding the daily taxi trips.
    """

    if not isinstance(taxi_trips, pd.DataFrame):
        raise TypeError("taxi_trips is not a valid pandas DataFrame.")
    
    taxi_trips.drop(["pickup_census_tract", "dropoff_census_tract", 
                     "pickup_centroid_location", "dropoff_centroid_location"], axis=1, inplace=True)

    taxi_trips.dropna(inplace=True)

    taxi_trips.rename(columns={"pickup_community_area": "pickup_community_area_id",
                            "dropoff_community_area": "dropoff_community_area_id" }, inplace=True)

    taxi_trips["datetime_for_weather"] = pd.to_datetime(taxi_trips["trip_start_timestamp"]).dt.floor("h")

    return taxi_trips

#### Company update codes

In [9]:
company_master = taxi_trips["company"].drop_duplicates().reset_index(drop=True)

company_master = pd.DataFrame(
    {
        "company_id": range(1, len(company_master) + 1),
        "company": company_master
    }
)

company_master.tail()

,company_id,company
28,29,U Taxicab
29,30,Tac - Blue Diamond Dispatch
30,31,2733 - 74600 Benny Jona
31,32,5167 - 71969 5167 Taxi Inc
32,33,Tac - Yellow Non Color


In [10]:
new_company_data = [
    {"company": "Koam Taxi Association"},
    {"company": "X"},
    {"company":"Y"}
]

new_company_mapping = pd.DataFrame(new_company_data)

new_company_mapping

,company
0,Koam Taxi Association
1,X
2,Y


In [11]:
company_max_id = company_master["company_id"].max()
company_max_id

np.int64(33)

In [12]:
new_companies_list = []

for company in new_company_mapping["company"].values:
    if company not in company_master["company"].values:
        new_companies_list.append(company)

# one line
new_companies_list = [company for company in new_company_mapping["company"].values if company not in company_master["company"].values]

new_companies_list

['X', 'Y']

In [13]:
new_companies_df = pd.DataFrame({
    "company_id": range(company_max_id + 1, company_max_id +len(new_companies_list) + 1),
    "company": new_companies_list
})

new_companies_df

,company_id,company
0,34,X
1,35,Y


In [14]:
updated_company_master = pd.concat([company_master, new_companies_df], ignore_index=True)

updated_company_master.tail()

,company_id,company
30,31,2733 - 74600 Benny Jona
31,32,5167 - 71969 5167 Taxi Inc
32,33,Tac - Yellow Non Color
33,34,X
34,35,Y


In [15]:
def update_company_master(taxi_trips: pd.DataFrame, company_master: pd.DataFrame) -> pd.DataFrame:
    """Extend the company master with new companies if there are new companies.

    Parameters
    ----------
    taxi_trips : pd.DataFrame
        DataFrame holding the daily data trips.
    company_master : pd.DataFrame
        DataFrame holding the company_master data.

    Returns
    -------
    pd.DataFrame
        The updated company_master data, if new companies are in the taxi data, they will be loaded to it.
    """

    company_max_id = company_master["company_id"].max()
    new_companies_list = [company for company in taxi_trips["company"].values if company not in company_master["company"].values]
    new_companies_df = pd.DataFrame({
        "company_id": range(company_max_id + 1, company_max_id + len(new_companies_list) + 1),
        "company": new_companies_list
    })

    new_companies_df

    updated_company_master = pd.concat([company_master, new_companies_df], ignore_index=True)

    return updated_company_master

In [16]:
taxi_trips_company_only = pd.DataFrame({
    "company_id": [1, 2, 3],
    "company": ["Koam Taxi Association", "X", "Y"]
})

taxi_trips_company_only

,company_id,company
0,1,Koam Taxi Association
1,2,X
2,3,Y


In [17]:
updated_company_master = update_company_master(taxi_trips=taxi_trips_company_only, company_master=company_master)

In [18]:
updated_company_master.tail()

,company_id,company
30,31,2733 - 74600 Benny Jona
31,32,5167 - 71969 5167 Taxi Inc
32,33,Tac - Yellow Non Color
33,34,X
34,35,Y


#### payment_type_master codes

In [19]:
payment_type_master = taxi_trips["payment_type"].drop_duplicates().reset_index(drop=True)

payment_type_master = pd.DataFrame(
    {
        "payment_type_id": range(1, len(payment_type_master) + 1),
        "payment_type": payment_type_master
    }
)

taxi_trips_payment_type_only = pd.DataFrame({
    "payment_type_id": [1, 2, 3],
    "payment_type": ["Credit Card", "X", "Y"]
})

taxi_trips_payment_type_only


,payment_type_id,payment_type
0,1,Credit Card
1,2,X
2,3,Y


In [20]:
def update_payment_type_master(taxi_trips: pd.DataFrame, payment_type_master: pd.DataFrame) -> pd.DataFrame:
    """Extend the payment type master with new payment types if there are new payment types.

    Parameters
    ----------
    taxi_trips : pd.DataFrame
        DataFrame holding the daily data trips.
    payment_type_master : pd.DataFrame
        DataFrame holding the company_master data.

    Returns
    -------
    pd.DataFrame
        The updated payment_type_master data, if new payment types are in the taxi data, they will be loaded to it.
    """

    payment_type_max_id = payment_type_master["payment_type_id"].max()

    new_payment_types_list = [payment_type for payment_type in taxi_trips["payment_type"].values if payment_type not in payment_type_master["payment_type"].values]
    new_payment_type_df = pd.DataFrame({
        "payment_type_id": range(payment_type_max_id + 1, payment_type_max_id + len(new_payment_types_list) + 1),
        "payment_type": new_payment_types_list
    })

    updated_payment_type_master = pd.concat([payment_type_master, new_payment_type_df], ignore_index=True)

    return updated_payment_type_master

In [21]:
updated_payment_type_master = update_payment_type_master(taxi_trips=taxi_trips_payment_type_only, payment_type_master=payment_type_master)

In [22]:
updated_payment_type_master

,payment_type_id,payment_type
0,1,Credit Card
1,2,Cash
2,3,Mobile
3,4,Unknown
4,5,Prcard
5,6,No Charge
6,7,Dispute
7,8,X
8,9,Y


### Creating a generic master table function

In [23]:
def update_master(taxi_trips: pd.DataFrame, master: pd.DataFrame, id_column: str, value_column: str) -> pd.DataFrame:
    """Extend the master DataFrame with new values if there are any.

    Parameters
    ----------
    taxi_trips : pd.DataFrame
        DataFrame holding the daily data trips.
    master : pd.DataFrame
        DataFrame holding the master data.
    id_column : str
        The id column of the master DataFrame.
    value_column : str
        Name of the column in master_df containing the values.

    Returns
    -------
    pd.DataFrame
        The updated master data, if new values are in the taxi data, they will be loaded to it.
    """

    max_id = master[id_column].max()

    new_values_list = [value for value in taxi_trips[value_column].values if value not in master[value_column].values]
    new_values_df = pd.DataFrame({
        id_column: range(max_id + 1, max_id + len(new_values_list) + 1),
        value_column: new_values_list
    })

    updated_master = pd.concat([master, new_values_df], ignore_index=True)

    return updated_master

In [24]:
test_payment_type_master = update_master(taxi_trips=taxi_trips_payment_type_only, master=payment_type_master, id_column ="payment_type_id", value_column = "payment_type")

In [25]:
test_payment_type_master

,payment_type_id,payment_type
0,1,Credit Card
1,2,Cash
2,3,Mobile
3,4,Unknown
4,5,Prcard
5,6,No Charge
6,7,Dispute
7,8,X
8,9,Y


In [26]:
test_company_master = update_master(taxi_trips=taxi_trips_company_only, master=company_master, id_column ="company_id", value_column = "company")

In [27]:
test_company_master.tail()

,company_id,company
30,31,2733 - 74600 Benny Jona
31,32,5167 - 71969 5167 Taxi Inc
32,33,Tac - Yellow Non Color
33,34,X
34,35,Y


### update taxi_trips with the most recent company_master and payment_type master function

In [28]:
def update_taxi_trips_with_master_data(taxi_trips: pd.DataFrame, payment_type_master: pd.DataFrame, company_master: pd.DataFrame) -> pd.DataFrame:
    """Upgrade the taxi_trips DataFrame with the company_master and payment_type_master ids, and delete the string columns.

    Parameters
    ----------
    taxi_trips : pd.DataFrame
        The DataFrame with the daily taxi trips.
    payment_type_master : pd.DataFrame
        The payment_type master table.
    company_master : pd.DataFrame
        The company master table.

    Returns
    -------
    pd.DataFrame
        The taxi_trips data, with only payment_type_id and company_id, without company or payment_type values.
    """

    taxi_trips_id = taxi_trips.merge(payment_type_master, on ="payment_type")
    taxi_trips_id = taxi_trips_id.merge(company_master, on="company")

    taxi_trips_id.drop(["payment_type", "company"], axis=1, inplace=True)

    return taxi_trips_id

In [29]:
taxi_trips_id = update_taxi_trips_with_master_data(taxi_trips=taxi_trips, payment_type_master=payment_type_master, company_master=company_master)

taxi_trips_id.sample(5)

,trip_id,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_community_area_id,dropoff_community_area_id,fare,tips,tolls,extras,trip_total,pickup_centroid_latitude,pickup_centroid_longitude,dropoff_centroid_latitude,dropoff_centroid_longitude,datetime_for_weather,payment_type_id,company_id
14874,700a6d5d476f3f6b504be57b72540ba349db0b02,7805fc9f8d756c8798e3b143635938b447596d082e3113...,2024-10-19T06:30:00.000,2024-10-19T07:00:00.000,1849,16.04,7,56,38.58,8.49,0,0,47.07,41.922686284,-87.649488729,41.79259236,-87.769615453,2024-10-19 06:00:00,3,9
14888,4768c4e5e0ed903626bdf43cdff420e9ef306ac9,72fd2a97a38a77919c31eb8875711aefb94f776ceb4fef...,2024-10-19T06:30:00.000,2024-10-19T06:45:00.000,701,5.8,38,42,16.75,0,0,0,16.75,41.812948939,-87.617859676,41.77887686,-87.594925439,2024-10-19 06:00:00,5,1
4963,6475981e24ccc820659976c3002a1345428cd8cd,794390dc445ec6a53c33b99dbf98a76b9a0c91194ae17d...,2024-10-19T17:45:00.000,2024-10-19T18:15:00.000,1836,2.01,8,32,14.5,3.75,0,0,18.75,41.905857769,-87.630865027,41.877406123,-87.621971652,2024-10-19 17:00:00,1,2
4610,94faf5e3c2b62f1f7240497e9de7805e07b38efe,13016372e777da1289d557edbe4ce2be8a68e77bc64768...,2024-10-19T18:00:00.000,2024-10-19T18:30:00.000,1701,1.78,32,8,14,3,0,0,17.5,41.870607372,-87.622172937,41.89321636,-87.63784421,2024-10-19 18:00:00,1,5
5404,f2a32ce5b3f109db7621727bcfb301dd8bd54cd6,78fb99d332dd664846f7934b7292dbb205641674541ff2...,2024-10-19T17:15:00.000,2024-10-19T17:30:00.000,480,1,3,3,6.5,0,0,0,6.5,41.96581197,-87.655878786,41.96581197,-87.655878786,2024-10-19 17:00:00,4,6


### weather transformations function

In [30]:
def transform_weather_data(weather_data: json) -> pd.DataFrame:
    """Make transformations on the daily weather api resonse.

    Parameters
    ----------
    weather_data : json
        The daily weather data from the Open Meteo API. 

    Returns
    -------
    pd.DataFrame
        The DataFrame representation of the data.
    """
    weather_data_filtered = {
        "datetime": weather_data["hourly"]["time"],
        "temperature": weather_data["hourly"]["temperature_2m"],
        "wind_speed": weather_data["hourly"]["wind_speed_10m"],
        "rain": weather_data["hourly"]["rain"],
        "precipitation": weather_data["hourly"]["precipitation"],
    }

    weather_df = pd.DataFrame(weather_data_filtered)

    weather_df["datetime"] = pd.to_datetime(weather_df["datetime"])

    return weather_df

In [31]:
# Test

current_datetime = datetime.now() - relativedelta(months=2)
formatted_datetime = current_datetime.strftime("%Y-%m-%d")

url = "https://archive-api.open-meteo.com/v1/era5"

params = {
    "latitude": 52.52,
    "longitude": 13.41,
    "start_date": formatted_datetime,
    "end_date": formatted_datetime,
    "hourly": "temperature_2m,wind_speed_10m,rain,precipitation"
}

response = requests.get(url, params = params)

weather_data= response.json()

weather_data_df = transform_weather_data(weather_data)

In [32]:
weather_data_df.head()

,datetime,temperature,wind_speed,rain,precipitation
0,2024-10-19 00:00:00,11.0,15.3,0.0,0.0
1,2024-10-19 01:00:00,10.6,13.3,0.0,0.0
2,2024-10-19 02:00:00,10.5,11.9,0.0,0.0
3,2024-10-19 03:00:00,10.6,11.9,0.0,0.0
4,2024-10-19 04:00:00,10.3,12.3,0.0,0.0
